# Tutorial on 1-D modelling of river long profiles
### Detachment-limited river
A river whose profile is given in the form of an excel file has a clear knickpoint around 12 km. In the absence of further structural, geologic, or tectonic information, we will try to reproduce this knickpoint under different scenarios assuming a detachment-limited model, where incision rate $E=KA^{m}S^{n}$ with $n$ and $m$ fixed respectively at 1 and 0.5.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
lp = pd.read_excel('~/riverProfile_tutorial/River_profile_new.xlsx')
plt.figure()
plt.plot(lp["X (km)"], lp["Z (m)"], 'b-')
plt.xlabel('Distance (km)')
plt.ylabel('Elevation (m)')
plt.xlim(min(lp["X (km)"]), max(lp["X (km)"]))
plt.ylim(min(lp["Z (m)"]), max(lp["Z (m)"]))
plt.title('River Profile')

In [ ]:
# Plotting slope vs Area
slopes = np.gradient(lp["Z (m)"], lp["X (km)"] * 1000) * (-1) # Since we are going down the river channel
plt.figure()
plt.scatter(lp["Area (km2)"], slopes, marker = '^', facecolor = "None", edgecolor = 'b')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Area (km2)')
plt.ylabel('Slope (m/m)')
plt.title('Slope-Area')

Since the river system is a detachment-limited system, the governing equation can be written as follows:
$$
\frac{\partial z}{\partial t}=U-K{A}^{m}{S}^{n}
\tag{1}
$$
At steady state Equation (1) can be rearranged and a relation between slope and area can be established, which takes the following form:
$$
S=\left(\frac{U}{K} \right)^\frac{1}{n}A^{-\frac{m}{n}}
\tag{2}
$$
If we take the natural log of both sides of Equation (2), we have an equation of a straight line between $\log{S}$ and $\log{A}$. The intercept of that line will be $\frac{1}{n}\log\left(\frac{U}{K} \right)$.</br></br>
A clear knickpoint is present along the river profile, which divides the river profile into two stretches. The first exercise is to have an intuitive estimate of the uplift rate for the two stretches of the river profile. This indicates possible transient condition of the river profile. The evolution of the river profile can be affected by different uplift rates (can vary in space or in time) and variable lithology. </br></br>

### 1. Steady-state uplift rate with spatial variations of uplift rates, and uniform erodibility coefficient K.
In this scenario, we will assume a constant erodibility factor $\left( K \right)$ as $2.10^{-5}\;yr^{-1}$. First, we will separate both reaches of the river by indexing the `lp` data series. To get an estimate of the uplift, we need the value of the intercept (where $\log S$ is zero). Since we know the value of $K$ in Equation (1), we can calculate an approximate uplift rate for both the stretches from the y-intercept of the slope-area plot. We have to carry out the exercise separately for the reaches separated by the knickpoint.

In [ ]:
# the approximate uplift rate in the two main segments of the river
# Divide the river profile into 2 reaches (i.e. upto 12 km, and 12-20 km)
p1 = 12
lp_1 = lp[lp["X (km)"] < p1]
lp_2 = lp[lp["X (km)"] >= p1]
# Slope vs Area plot for the two reaches
S1 = slopes[lp["X (km)"] < p1]
A1 = lp[lp["X (km)"] < p1]["Area (km2)"]
S2 = slopes[lp["X (km)"] >= p1]
A2 = lp[lp["X (km)"] >= p1]["Area (km2)"]
# PLot the river profile and the slope-area plot in a single plot. 
# Slope-Area plot will plot log(slope) vs log(Area) for the two stretches.
plt.figure()
plt.subplot(2, 2, 1)
plt.plot(lp_1["X (km)"], lp_1["Z (m)"], 'k-')
plt.subplot(2, 2, 2)
plt.plot(lp_2["X (km)"], lp_2["Z (m)"], 'k-')
plt.subplot(2, 2, 3)
plt.scatter(np.log(A1), np.log(S1), marker = '^', facecolor = "None", edgecolor = 'k')
plt.subplot(2, 2, 4)
plt.scatter(np.log(A2), np.log(S2), marker = '^', facecolor = "None", edgecolor = 'k')

From the plot above, observe both the slope-area plot on the log-log scale. These approximate estimates of the uplift rates will give us a clue about the range of uplift rates that the river profile has undergone. Next, we will try to get the time-dependent solution of the river profile by solving Equation (1) using the  <b>Finite Difference</b> technique.

In [ ]:
# River profile
Xriv = lp["X (km)"] * 1000   # distance in m
Zriv = lp["Z (m)"]
Area = lp["Area (km2)"] * 1e6   # area in m2
plt.figure()
plt.plot(Xriv / 1000, Zriv, 'b', linewidth=2)
plt.xlabel('distance (km)')
plt.ylabel('elevation (m)')

#### Parameter definition

In [ ]:
# ------ Incision Parameters  ----------
m = 0.5 #incision law area exponent 
n = 1 #incision law slope exponent 

ka = 6.69 # area length coefficient (Hack 1957 _ L en m et A en m2)
h = 1.67 # réciproque de l'exposant de Hack (valeur observée, Hack 1957)
theta = h*m/n # index of concavity between z and x (not between z and A)

# ------ River spatial characteristics ------
    
L = 20000 # river length (m) - CAN BE MODIFIED 
zi = 0    # z(L) = altitude of the base level (m) - CAN BE MODIFIED 
Asource=0.5e6   # area of the  sources in m2
xmin=(Asource/ka)**(1/h)  # distance between the crest and the source = limit of application of the fluvial incision law
dx = 50 # spatial step (m) - CAN BE MODIFIED>. The evolution equation will be numerically solved on this spatial scale

x = np.arange(0,L,dx)   # downstream distance vector (m). Discretization of spatial domain
nx = len(x)
n0 = np.where(x<xmin)[0][-1]  # index of the start of the fluvial drain. This is the last position of the spatial domain where we have channels.

# ------ Time characteristics ------
T_end = int(4e5) # duration of the run in year 300 kyr  
dt = 100    # time step in year
time = np.arange(0,T_end,dt)  # time vector. Discretization of the time domain
nt = len(time)

# ------- Uplift, erodibility coefficient and their variations ------ 
#  spatial variations in local uplift rate or erodibility
Xt = [1, 12000]
ntX = len(Xt)  # number of spatial transition
Up = [2, 5] # uplift rate in mm/yr _ Must be the same length as Xt. Uplift rate in the 2 separate domain
Up = [u / 1000 for u in Up]    # uplift rate in m/yr
K = [2E-5, 2E-5] # erosion coefficient related to rock erodibility _ Must be same length as Xt

#  temporal variations in regional uplift rate. Please do this experiment.
Timing = [0, 1.7E+5]  # timing in yr of the change in regional uplift rate 
ntT = len(Timing)  # number of temporal transition
Ufac = [1, 1]  # uplift temporal change factor _ Must be same length as Xt

# index linked to the spatial variations
nxt1 = np.where(x<Xt[1])[0][-1]

UpV = np.zeros((nx,))   # spatial vector of the local uplift value
UpV[0 : nxt1] = Up[0]  
UpV[nxt1 : nx] = Up[1]
KV = np.zeros((nx,))   # spatial vector of the local uplift value
KV[0 : nxt1] = K[0]  
KV[nxt1 : nx] = K[1]

# For temporal variability in the model params
it1 = np.where(time < Timing[1])[0][-1]
UfacV = np.zeros((nt,))   # spatial vector of the local uplift value
# Assign temporal uplift change factor to each time window
UfacV[0 : it1] = Ufac[0]  
UfacV[it1 : nt] = Ufac[1]

#### Initial profile = arbitrary

In [ ]:
# Initial state parameter
Ui = 0.001 # rock uplift initial (m/an) 
Ki = 0.00002
z0 = np.zeros((nx,))
# Steady state solution (z(x) when dz/dt = 0) with the initial parameters
z0[n0 : nx] = zi + ((Ui / Ki / (ka ** m)) ** (1 / n)) * ((L ** (1 - theta) - x[n0 : nx] ** (1 - theta)) / (1 - theta))  
# initial profile 
plt.plot(x[n0 : nx] / 1000, z0[n0 : nx], 'limegreen', linewidth = 2)
# Observed (current) profile
plt.plot(Xriv / 1000, Zriv, 'b', linewidth=2)
plt.xlim(0, 20)
plt.ylim(0,)
plt.xlabel('distance (km)')
plt.ylabel('elevation (m)')

#### Transient profiles

In [ ]:
# For animation purposes
from IPython.display import display, clear_output
nbr = 10 # number of transient profile for display _ CAN BE MODIFIED 

# area interpolation to define the drainage area in any point of the
# numeric river
AreaX = np.interp(x, Xriv, Area)
AreaX = np.maximum(AreaX, Asource/2)  # to avoid to have negative area

## ----------  finite difference solution
iC = round(nt / nbr)
zz_num = z0.copy() # creation of the vector containing the elevation of the transient profile
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) 
ax.plot(Xriv / 1000, Zriv, 'b', linewidth=2, alpha = 0.5)
ax.set_xlabel('distance (km)')
ax.set_ylabel('elevation (m)')
ax.set_ylim(0,)
ax.plot(x[n0 : nx] / 1000, z0[n0 : nx], 'limegreen', linewidth = 2, alpha = 0.5)

for i in range(nt):    # time iteration
    S = np.maximum(0, -np.diff(zz_num)) / dx   # gradient of the river ; in x=L (index nx) S=0 permanently
                                   # the max value prevents to get negative slopes
    zz_num[0 : nx-1] = zz_num[0 : nx-1] + dt * ((UfacV[i] * UpV[0 : nx-1]) - (KV[0 : nx-1]*(AreaX[0 : nx-1]**m)*(S**n)))
    
    # --------
    if (i % iC == 0):
        ax.plot(x[n0:nx] / 1000, zz_num[n0 : nx], color = 'gray')  # display of the intermediate profiles
        display(fig)
        clear_output(wait = True)
        plt.pause(1.0)
        #plt.pause(0.1) # After every plot the program pauses for 0.1 sec
    
#plt.plot(x[n0:nx] / 1000, zz_num[n0 : nx], color = 'gray')

#### Setting up a simple inversion model by error function (MSE) minimization
In this exercise (for scenario 1), we will try to find the best estimate for the two uplift rates (U1 and U2). To do that, we will use a simple optimization routine, where we first sample the model parameter space ($U1-U2$ space) within specific ranges (1 mm/yr to 7 mm/yr). For each combination of the model parameters we run our forward model like in previous step for 300 kyr and calculate an error value defined as follows:
$$
e=\frac{1}{N}\sum\left(z_{model}-z_{obs}\right)^{2}
\tag{3}
$$
$N$ is the length of the data (elevation of the river profile) vector. In the end we will find the index for the mimimum value of the error matrix to get the best estimate of the uplift rates U1 and U2.

In [ ]:
# Define an error function. Only includes data error. Other error forms can be included.
def error_function(z_obs, z_model):
    e = (1 / len(z_obs)) * sum((z_model - z_obs) ** 2)
    return e

In [ ]:
# Define model parameter space
Up1 = np.arange(1.0, 7.3, 0.3, dtype='float') # Sampling interval (0.1) can be increased to have a lesser sample size of the model parameters.
# Decreasing the sampling interval will increase the fidelity of your inversion routine. But it will take more number iterations and hence more time.
Up2 = np.arange(1.0, 7.3, 0.3, dtype='float')
# Initialize an error matrix
e_matrix = np.zeros((len(Up1), len(Up2)))
# Run the forward model for every possible scenario (a brute-force inversion approach)
for (r_idx, u1) in enumerate(Up1):
    for (c_idx, u2) in enumerate(Up2):
        Up = [u1, u2]
        Up = [u / 1000 for u in Up]    # uplift rate in m/yr
        K = [2E-5, 2E-5]
        Xt = [1, 12000]
        ntX = len(Xt)  # number of spatial transition
        UpV = np.zeros((nx,))   # spatial vector of the local uplift value
        UpV[0 : nxt1] = Up[0]  
        # UpV[nxt1 : nxt2]=Up[1]
        UpV[nxt1 : nx]=Up[1]
        
        zz_num = z0.copy()
        for i in range(nt):    # time iteration
            S = np.maximum(0, -np.diff(zz_num)) / dx
            zz_num[0 : nx-1] = zz_num[0 : nx-1] + dt * ((UfacV[i] * UpV[0 : nx-1]) - (KV[0 : nx-1]*(AreaX[0 : nx-1]**m)*(S**n)))
        z_mod = np.interp(Xriv, x, zz_num)
        e = error_function(Zriv, z_mod)
        e_matrix[r_idx, c_idx] = e

In [ ]:
# Visualization of the error matrix
plt.figure()
plt.imshow(np.log(e_matrix), extent=[min(Up2), max(Up2), min(Up1), max(Up1)], cmap= "seismic")
plt.colorbar(extend='both', 
             orientation='vertical', 
             aspect = 30, shrink=1.0, pad=0.06)
plt.xlabel('U2 (mm/yr)')
plt.ylabel('U1 (mm/yr)')
plt.tight_layout()

In [ ]:
# Estimation of the unknown model parameters. Since we only have two model params here, we can easily visualize it. 
# As the number of model params increase, we have to find the index instead and find the model parameter estimates.
idx_min_err = np.where(e_matrix == np.min(e_matrix))
U1 = Up1[idx_min_err[0]][0]
U2 = Up2[idx_min_err[1]][0]
print(f'U1 is {U1:.2f} mm/yr, U2 is {U2:.2f} mm/yr')

### 2. Steady-state uplift rate with spatial variations of erodibility coefficient K, and uniform uplift rates.
In this scenario, impose a uniform uplift rate, try 2 values for K, run the program. Modify the 2 values until obtaining a final profile that fits the observed river profile. Do you observe the same phenomenon than during the runs of scenario 1? Why?

### 3. Unsteady uplift rate (temporal variation) with spatially uniform erodibility coefficient K and uplift rates.
The unsteady scenario propose to explore a pulse of increased uplift between Timing[1] and nt (total duration of the forward model). The increase of uplift can be defined by the U_factor (Ufac). Impose a uniform uplift rate and coefficient K, and play with the uplift increase factor and the Timing, in order again to match, if possible, the observed river profile. In order to avoid random scenario, it is possible to roughly compute the wave speed of the receding knickpoint $\left(KA^mS^{n-1}\right)$, and according to the distance between the base level (x=L) and the knickpoint to estimate the Timing of the uplift perturbation.